In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./SchemeData1908231158SS.csv')

In [3]:
data.head()

,AMC,Code,Scheme Name,Scheme Type,Scheme Category,Scheme NAV Name,Scheme Minimum Amount,Launch Date,Closure Date,ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment
0,Aditya Birla Sun Life AMC Limited,100033,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,5000,24-Feb-1995,24-Feb-1995,INF209K01165
1,Aditya Birla Sun Life AMC Limited,100034,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -R...,5000,24-Feb-1995,24-Feb-1995,INF209K01157INF209K01CE5
2,Aditya Birla Sun Life AMC Limited,119433,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -D...,5000,24-Feb-1995,24-Feb-1995,INF209K01VG0INF209K01VI6
3,Aditya Birla Sun Life AMC Limited,119436,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,5000,24-Feb-1995,24-Feb-1995,INF209K01VH8
4,Aditya Birla Sun Life AMC Limited,100037,Aditya Birla Sun Life Income Fund,Open Ended,Debt Scheme - Medium to Long Duration Fund,Aditya Birla Sun Life Income Fund - Regular - ...,5000,01-Jan-1994,01-Jan-1994,INF209K01587INF209K01CY3


In [4]:
bubbleData = pd.DataFrame({'AMC': data['AMC'], "AMFICode": data['Code'], 'Scheme Name': data['Scheme Name'], 'Scheme Type': data['Scheme Type'], 'AMFI Scheme Category': data['Scheme Category'], 'Scheme NAV Name': data['Scheme NAV Name']})

In [5]:
def scheme_category(scheme):
    list_ = [i.replace("Scheme","").replace("&", "and").strip() for i in scheme.split(' - ')]
    if len(list_) < 2:
        return ["", list_[0]]
    return list_

In [6]:
bubbleData[["Scheme Category", "Sub Category"]] = bubbleData['AMFI Scheme Category'].apply(scheme_category).apply(pd.Series)

In [7]:
bubbleData['Fundtype'] = bubbleData['Sub Category'].apply(lambda x: 'FoF' if 'FoF' in x else 'DE')

In [8]:
bubbleData['PlanType'] = bubbleData['Scheme NAV Name'].apply(lambda x: 'Direct' if 'direct' in x.lower() else 'Regular')

In [9]:
bubbleData['PlanType'].value_counts()

Regular    8083
Direct     6852
Name: PlanType, dtype: int64

In [10]:
def ISIN(x):
    isin_payout = ""
    isin_reinv = ""
    if '-' in x:
        x = x.split('-')
    else:
        x = x.split(' ')
    x = [i.strip() for i in x if len(i)>=12]
    if len(x)==1:
        x=x[0]
        if len(x)==24:
            isin_payout = x[0:12]
            isin_reinv = x[12:24]
        elif len(x)==12:
            isin_payout = x
    elif len(x)==2:
        if len(x[0]) == 12:
            isin_payout = x[0]
        if len(x[1]) == 12:
            isin_reinv = x[1]
    return [isin_payout, isin_reinv]

    return x

In [11]:
bubbleData[["ISIN Div Payout / ISIN Growth", "ISIN Div Reinvestment"]] = data['ISIN Div Payout/ ISIN GrowthISIN Div Reinvestment'].apply(str).apply(ISIN).apply(pd.Series)

In [12]:
bubbleData['Launch Date'] = data['Launch Date']
bubbleData['Closure Date'] = data[' Closure Date']

In [13]:
def amount(x):
    min_amount = ''
    x+=' '
    for i in range(len(x)):
        if x[i].isdigit():
            for k in x[i:]:
                if k.isdigit():
                    min_amount += k
                else:
                    return int(min_amount)
    return None

In [14]:
bubbleData['Scheme Minimum Amount'] = data['Scheme Minimum Amount'].apply(str).apply(amount)

In [27]:
bubbleData['isPrimary'] = bubbleData['Scheme NAV Name'].apply(lambda x: 'Y' if 'growth' in x.lower() or 'regular' in x.lower() else 'N')

In [28]:
bubbleData

,AMC,AMFICode,Scheme Name,Scheme Type,AMFI Scheme Category,Scheme NAV Name,Scheme Category,Sub Category,Fundtype,PlanType,ISIN Div Payout / ISIN Growth,ISIN Div Reinvestment,Launch Date,Closure Date,Scheme Minimum Amount,isPrimary
0,Aditya Birla Sun Life AMC Limited,100033,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,Equity,Large and Mid Cap Fund,DE,Regular,INF209K01165,,24-Feb-1995,24-Feb-1995,5000.0,Y
1,Aditya Birla Sun Life AMC Limited,100034,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -R...,Equity,Large and Mid Cap Fund,DE,Regular,INF209K01157,INF209K01CE5,24-Feb-1995,24-Feb-1995,5000.0,Y
2,Aditya Birla Sun Life AMC Limited,119433,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund -D...,Equity,Large and Mid Cap Fund,DE,Direct,INF209K01VG0,INF209K01VI6,24-Feb-1995,24-Feb-1995,5000.0,N
3,Aditya Birla Sun Life AMC Limited,119436,Aditya Birla Sun Life Equity Advantage Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Equity Advantage Fund - ...,Equity,Large and Mid Cap Fund,DE,Direct,INF209K01VH8,,24-Feb-1995,24-Feb-1995,5000.0,Y
4,Aditya Birla Sun Life AMC Limited,100037,Aditya Birla Sun Life Income Fund,Open Ended,Debt Scheme - Medium to Long Duration Fund,Aditya Birla Sun Life Income Fund - Regular - ...,Debt,Medium to Long Duration Fund,DE,Regular,INF209K01587,INF209K01CY3,01-Jan-1994,01-Jan-1994,5000.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14930,Navi AMC Limited,152062,NAVI S&P BSE SENSEX INDEX FUND,Open Ended,Other Scheme - Index Funds,NAVI S&P BSE SENSEX INDEX FUND - DIRECT PLAN -...,Other,Index Funds,DE,Direct,INF959L01HK9,,21-Aug-2023,NaN,10.0,Y
14931,Kotak Mahindra Asset Management Company Limited.,152063,Kotak Multi Asset Allocation Fund,Open Ended,Hybrid Scheme - Multi Asset Allocation,Kotak Multi Asset Allocation Fund - Direct Pla...,Hybrid,Multi Asset Allocation,DE,Direct,INF174KA1PE2,INF174KA1PF9,31-Aug-2023,NaN,5000.0,N
14932,Kotak Mahindra Asset Management Company Limited.,152064,Kotak Multi Asset Allocation Fund,Open Ended,Hybrid Scheme - Multi Asset Allocation,Kotak Multi Asset Allocation Fund - Direct Pla...,Hybrid,Multi Asset Allocation,DE,Direct,INF174KA1PD4,,31-Aug-2023,NaN,5000.0,Y
14933,Kotak Mahindra Asset Management Company Limited.,152065,Kotak Multi Asset Allocation Fund,Open Ended,Hybrid Scheme - Multi Asset Allocation,Kotak Multi Asset Allocation Fund - Regular Pl...,Hybrid,Multi Asset Allocation,DE,Regular,INF174KA1PA0,,31-Aug-2023,NaN,5000.0,Y


In [30]:
bubbleData.to_csv('Scheme-Master.csv', index=False)